In [168]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from typing import List

In [153]:
caminho_base = '../data/processed/df_processed_v2.parquet'

In [154]:
base_completa = pd.read_parquet(caminho_base)

In [193]:
base_completa

,NOME_AEROPORTO_ORIGEM,CIDADE_AEROPORTO_ORIGEM,UF_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NUMERO_VOO,CODIGO_TIPO_LINHA,SIGLA_ICAO_AEROPORTO_ORIGEM,SIGLA_ICAO_AEROPORTO_DESTINO,PARTIDA_PREVISTA,PARTIDA_REAL,...,INDICE_SEMANA_CHEGADA_REAL,INDICE_SEMANA_CHEGADA_PREVISTA,FAIXA_ATRASO_PARTIDA,FAIXA_ATRASO_CHEGADA,FLAG_ATRASO_PARTIDA,FLAG_ATRASO_CHEGADA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,FLAG_FIM_SEMANA_PARTIDA_REAL,FLAG_FIM_SEMANA_CHEGADA_PREVISTA,FLAG_FIM_SEMANA_CHEGADA_REAL
0,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-02 00:05:00,2022-01-01 23:35:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
1,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-09 01:00:00,2022-01-09 00:48:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
2,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-23 11:05:00,2022-01-23 10:54:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
3,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-01-30 11:05:00,2022-01-30 11:01:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
4,BAURU/AREALVA,AREALVA,SP,AZU,2455,N,SBAE,SBPS,2022-02-26 23:00:00,2022-02-26 22:56:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717352,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,1900-01-01 00:00:00,2022-12-15 14:34:00,...,3,0,Atraso de mais de 4 horas,Atraso de mais de 4 horas,Vôo no horário,Vôo no horário,Dia Útil,Dia Útil,Dia Útil,Dia Útil
717353,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,2022-12-18 15:35:00,2022-12-18 15:41:00,...,6,6,Atraso de até 30 minutos,No horário,Vôo no horário,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana
717354,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,1900-01-01 00:00:00,2022-12-22 14:46:00,...,3,0,Atraso de mais de 4 horas,Atraso de mais de 4 horas,Vôo no horário,Vôo no horário,Dia Útil,Dia Útil,Dia Útil,Dia Útil
717355,DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,BA,ABJ,1416,N,SBSV,SIRI,2022-12-25 15:35:00,2022-12-25 15:11:00,...,6,6,No horário,No horário,Vôo Atrasado,Vôo Atrasado,Fim de Semana,Fim de Semana,Fim de Semana,Fim de Semana


- Situação Vôo

In [221]:
def obter_situacao_voo(
        sigla_aeroporto: str,
        mes_partida_prevista: List[int],
        codigo_tipo_linha: str, 
        sigla_empresa: str = None,
       
):
    if sigla_empresa is not None:
        query = (
            f'SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '
            f'and SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" '
            f'and MES_PARTIDA_PREVISTA in {mes_partida_prevista} ' 
            f'and CODIGO_TIPO_LINHA = {codigo_tipo_linha}'
        )
        colunas = [
            'SIGLA_ICAO_EMPRESA_AEREA', 
            'SITUACAO_VOO', 
            'MES_PARTIDA_PREVISTA', 
            'NOME_MES_PARTIDA_PREVISTA', 
            'SIGLA_ICAO_AEROPORTO_ORIGEM'
        ]
        
    else:
        query = (
            f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" '
            f' and MES_PARTIDA_PREVISTA in {mes_partida_prevista} '  # TAM SBRP 1
            f'and CODIGO_TIPO_LINHA = {codigo_tipo_linha}'
        )
        colunas = [
            'SITUACAO_VOO', 
            'MES_PARTIDA_PREVISTA', 
            'NOME_MES_PARTIDA_PREVISTA', 
            'SIGLA_ICAO_AEROPORTO_ORIGEM'
         ]

    if sigla_empresa is not None:
        query = (
            f'SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa}" '
            f'and SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" '
            f'and MES_PARTIDA_PREVISTA in {mes_partida_prevista} '  # TAM SBRP 1
            f'and CODIGO_TIPO_LINHA = {codigo_tipo_linha}'
        )
        
    else:
        query = (
            f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" '
            f' and MES_PARTIDA_PREVISTA in {mes_partida_prevista} '  # TAM SBRP 1
        )
         
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    total = dataframe.query(query)[colunas].value_counts()
    percentual = dataframe.query(query)[colunas].value_counts( normalize=True) * 100
    dataframe = pd.concat([total, percentual], axis=1).reset_index()
    dataframe['count'] = dataframe['count'].astype('int32')
    dataframe['proportion'] = dataframe['proportion'].astype('float32')
    dataframe['proportion'] = dataframe['proportion'].round(2)
    dataframe = dataframe.sort_values(by=['SITUACAO_VOO', 'MES_PARTIDA_PREVISTA']) 
    return dataframe 

In [222]:
obter_situacao_voo(
    sigla_aeroporto='SBRP', 
    mes_partida_prevista=[1, 2], 
    codigo_tipo_linha='N'
)

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,count,proportion
2,CANCELADO,1,janeiro,SBRP,21,2.410000
3,CANCELADO,2,fevereiro,SBRP,18,2.060000
0,REALIZADO,1,janeiro,SBRP,584,66.970001
1,REALIZADO,2,fevereiro,SBRP,249,28.559999


- Desempenho da Situação do Voo

In [166]:
colunas = ['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM']
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [16]:
dataframe = dataframe.query('SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
            ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"'
            ' and SITUACAO_VOO == "REALIZADO" ' 
            )

dataframe.head()

,SIGLA_ICAO_EMPRESA_AEREA,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM
488380,TAM,REALIZADO,1,janeiro,SBRP
488381,TAM,REALIZADO,1,janeiro,SBRP
537779,TAM,REALIZADO,1,janeiro,SBRP
537780,TAM,REALIZADO,1,janeiro,SBRP
537781,TAM,REALIZADO,1,janeiro,SBRP


In [17]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 488380 to 680870
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   SIGLA_ICAO_EMPRESA_AEREA     345 non-null    string  
 1   SITUACAO_VOO                 345 non-null    category
 2   MES_PARTIDA_PREVISTA         345 non-null    int32   
 3   NOME_MES_PARTIDA_PREVISTA    345 non-null    category
 4   SIGLA_ICAO_AEROPORTO_ORIGEM  345 non-null    string  
dtypes: category(2), int32(1), string(2)
memory usage: 10.6 KB


In [18]:
dataframe['SITUACAO_VOO'] = dataframe['SITUACAO_VOO'].astype('string')
dataframe['NOME_MES_PARTIDA_PREVISTA'] = dataframe['NOME_MES_PARTIDA_PREVISTA'].astype('string')
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 488380 to 680870
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   SIGLA_ICAO_EMPRESA_AEREA     345 non-null    string
 1   SITUACAO_VOO                 345 non-null    string
 2   MES_PARTIDA_PREVISTA         345 non-null    int32 
 3   NOME_MES_PARTIDA_PREVISTA    345 non-null    string
 4   SIGLA_ICAO_AEROPORTO_ORIGEM  345 non-null    string
dtypes: int32(1), string(4)
memory usage: 14.8 KB


In [19]:
dataframe_gb = dataframe.groupby(['MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA', 'SITUACAO_VOO'], observed=False).agg(
    TOTAL_SITUACAO=('SITUACAO_VOO', 'count')
)
dataframe_gb.sort_values(by=['MES_PARTIDA_PREVISTA'], inplace=True)
dataframe_gb.reset_index(inplace=True)
dataframe_gb

,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SITUACAO_VOO,TOTAL_SITUACAO
0,1,janeiro,REALIZADO,41
1,2,fevereiro,REALIZADO,20
2,3,março,REALIZADO,23
3,4,abril,REALIZADO,21
4,5,maio,REALIZADO,20
5,6,junho,REALIZADO,22
6,7,julho,REALIZADO,21
7,8,agosto,REALIZADO,23
8,9,setembro,REALIZADO,23
9,10,outubro,REALIZADO,24


In [20]:
fig = px.bar(
    dataframe_gb, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_SITUACAO', 
    text_auto=True,
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Total de Vôos por aeroporto

In [21]:
colunas = ['NOME_AEROPORTO_ORIGEM', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM']
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [22]:
dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"')

,NOME_AEROPORTO_ORIGEM,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM
1929,LEITE LOPES,1,janeiro,SBRP
1930,LEITE LOPES,1,janeiro,SBRP
1931,LEITE LOPES,1,janeiro,SBRP
1932,LEITE LOPES,1,janeiro,SBRP
1933,LEITE LOPES,1,janeiro,SBRP
...,...,...,...,...
707275,LEITE LOPES,1,janeiro,SBRP
707479,LEITE LOPES,1,janeiro,SBRP
707480,LEITE LOPES,1,janeiro,SBRP
707481,LEITE LOPES,1,janeiro,SBRP


In [23]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 4 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   NOME_AEROPORTO_ORIGEM        717357 non-null  string  
 1   MES_PARTIDA_PREVISTA         717357 non-null  int32   
 2   NOME_MES_PARTIDA_PREVISTA    717357 non-null  category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  717357 non-null  string  
dtypes: category(1), int32(1), string(2)
memory usage: 14.4 MB


In [24]:
dataframe['NOME_MES_PARTIDA_PREVISTA'] = dataframe['NOME_MES_PARTIDA_PREVISTA'].astype('string')

In [25]:
total_voos =dataframe.groupby(['MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('NOME_AEROPORTO_ORIGEM', 'count')
)
total_voos.reset_index(inplace=True)
total_voos['TOTAL_VOOS'] = total_voos['TOTAL_VOOS'].astype('int32')


In [26]:
total_voos

,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,TOTAL_VOOS
0,1,janeiro,78523
1,2,fevereiro,46597
2,3,março,54542
3,4,abril,52739
4,5,maio,58631
5,6,junho,55320
6,7,julho,63666
7,8,agosto,60677
8,9,setembro,58683
9,10,outubro,61976


In [27]:
total_voos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   MES_PARTIDA_PREVISTA       12 non-null     int32 
 1   NOME_MES_PARTIDA_PREVISTA  12 non-null     string
 2   TOTAL_VOOS                 12 non-null     int32 
dtypes: int32(2), string(1)
memory usage: 320.0 bytes


In [28]:
fig = px.bar(
    total_voos, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    text_auto='0',
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Total de Vôos por compania e aeroporto por mês

In [29]:
base.columns

Index(['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM'],
      dtype='object')

In [30]:
colunas = [
        
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'MES_PARTIDA_PREVISTA',
        'NOME_MES_PARTIDA_PREVISTA',
        'NUMERO_VOO' ,
        'SIGLA_ICAO_EMPRESA_AEREA',
        'NOME_EMPRESA' 
]


dataframe = pd.read_parquet(caminho_base, columns=colunas)
dataframe = dataframe.query(
    ' SIGLA_ICAO_EMPRESA_AEREA == "TAM"  ' \
    ' and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"'\
)

In [31]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348 entries, 488380 to 680870
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   SIGLA_ICAO_AEROPORTO_ORIGEM  348 non-null    string  
 1   MES_PARTIDA_PREVISTA         348 non-null    int32   
 2   NOME_MES_PARTIDA_PREVISTA    348 non-null    category
 3   NUMERO_VOO                   348 non-null    string  
 4   SIGLA_ICAO_EMPRESA_AEREA     348 non-null    string  
 5   NOME_EMPRESA                 348 non-null    string  
dtypes: category(1), int32(1), string(4)
memory usage: 15.7 KB


In [32]:
dataframe['NOME_MES_PARTIDA_PREVISTA'] = dataframe['NOME_MES_PARTIDA_PREVISTA'].astype('string')

In [33]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348 entries, 488380 to 680870
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   SIGLA_ICAO_AEROPORTO_ORIGEM  348 non-null    string
 1   MES_PARTIDA_PREVISTA         348 non-null    int32 
 2   NOME_MES_PARTIDA_PREVISTA    348 non-null    string
 3   NUMERO_VOO                   348 non-null    string
 4   SIGLA_ICAO_EMPRESA_AEREA     348 non-null    string
 5   NOME_EMPRESA                 348 non-null    string
dtypes: int32(1), string(5)
memory usage: 17.7 KB


In [34]:
dataframe.head()

,SIGLA_ICAO_AEROPORTO_ORIGEM,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,NUMERO_VOO,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
488380,SBRP,1,janeiro,3867,TAM,TAM
488381,SBRP,1,janeiro,3867,TAM,TAM
537779,SBRP,1,janeiro,3992,TAM,TAM
537780,SBRP,1,janeiro,3992,TAM,TAM
537781,SBRP,1,janeiro,3992,TAM,TAM


In [35]:
dataframe_gb = dataframe.groupby([ 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('SIGLA_ICAO_EMPRESA_AEREA', 'count')
).reset_index()

In [36]:
dataframe_gb

,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,TOTAL_VOOS
0,1,janeiro,41
1,2,fevereiro,20
2,3,março,23
3,4,abril,21
4,5,maio,20
5,6,junho,22
6,7,julho,21
7,8,agosto,23
8,9,setembro,23
9,10,outubro,24


In [37]:
fig = px.bar(
    dataframe_gb, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    text_auto='0',
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Analise da variação percentual da SITUAÇÂO voo Filtrando por Aerporto origem por mês (deixe filtro facultativo por empresa)

In [38]:
base.columns

Index(['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM'],
      dtype='object')

In [39]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
]
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [40]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   SITUACAO_VOO                 717357 non-null  category
 1   MES_PARTIDA_PREVISTA         717357 non-null  int32   
 2   NOME_MES_PARTIDA_PREVISTA    717357 non-null  category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  717357 non-null  string  
 4   NOME_AEROPORTO_ORIGEM        717357 non-null  string  
dtypes: category(2), int32(1), string(2)
memory usage: 15.1 MB


In [41]:
dataframe.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM
0,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
1,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
2,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
3,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA
4,REALIZADO,2,fevereiro,SBAE,BAURU/AREALVA


In [42]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   SITUACAO_VOO                 717357 non-null  category
 1   MES_PARTIDA_PREVISTA         717357 non-null  int32   
 2   NOME_MES_PARTIDA_PREVISTA    717357 non-null  category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  717357 non-null  string  
 4   NOME_AEROPORTO_ORIGEM        717357 non-null  string  
dtypes: category(2), int32(1), string(2)
memory usage: 15.1 MB


In [43]:
dataframe[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']] = dataframe[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']].astype('string')

In [44]:
dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"')
dataframe

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM
1929,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1930,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1931,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1932,REALIZADO,1,janeiro,SBRP,LEITE LOPES
1933,REALIZADO,1,janeiro,SBRP,LEITE LOPES
...,...,...,...,...,...
707275,REALIZADO,1,janeiro,SBRP,LEITE LOPES
707479,REALIZADO,1,janeiro,SBRP,LEITE LOPES
707480,REALIZADO,1,janeiro,SBRP,LEITE LOPES
707481,REALIZADO,1,janeiro,SBRP,LEITE LOPES


In [45]:
dataframe_gb =dataframe.groupby(['NOME_MES_PARTIDA_PREVISTA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('SITUACAO_VOO', 'count')
)

dataframe_gb.reset_index(inplace=True)
dataframe_gb.sort_values(by=['MES_PARTIDA_PREVISTA'], inplace=True)
dataframe_gb['TOTAL_VOOS'] = dataframe_gb['TOTAL_VOOS'].astype('int32')

In [46]:
dataframe_gb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 8 to 4
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   NOME_MES_PARTIDA_PREVISTA  24 non-null     string
 1   SITUACAO_VOO               24 non-null     string
 2   MES_PARTIDA_PREVISTA       24 non-null     int32 
 3   TOTAL_VOOS                 24 non-null     int32 
dtypes: int32(2), string(2)
memory usage: 768.0 bytes


In [47]:
dataframe_gb

,NOME_MES_PARTIDA_PREVISTA,SITUACAO_VOO,MES_PARTIDA_PREVISTA,TOTAL_VOOS
8,janeiro,CANCELADO,1,21
9,janeiro,REALIZADO,1,584
6,fevereiro,CANCELADO,2,18
7,fevereiro,REALIZADO,2,249
17,março,REALIZADO,3,345
16,março,CANCELADO,3,16
0,abril,CANCELADO,4,3
1,abril,REALIZADO,4,381
15,maio,REALIZADO,5,433
14,maio,CANCELADO,5,14


In [48]:
fig = px.bar(
    dataframe_gb, 
    x='NOME_MES_PARTIDA_PREVISTA', 
    y='TOTAL_VOOS', 
    color='SITUACAO_VOO',
    barmode='group',
    text_auto=True
)
fig.update_traces(
    textposition='outside'
)
fig.show()

- Analise da variação percentual da SITUAÇÂO voo Filtrando por Aerporto origem por mês compare com o mês anterior (Talvez com vísual cartões) deixe filtro facultativo por empresa)

In [49]:
base.columns

Index(['SIGLA_ICAO_EMPRESA_AEREA', 'SITUACAO_VOO', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'SIGLA_ICAO_AEROPORTO_ORIGEM'],
      dtype='object')

In [50]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA'

]
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [51]:
dataframe.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
0,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
1,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
2,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
3,REALIZADO,1,janeiro,SBAE,BAURU/AREALVA,AZU,AZUL
4,REALIZADO,2,fevereiro,SBAE,BAURU/AREALVA,AZU,AZUL


In [52]:
dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"  and SIGLA_ICAO_EMPRESA_AEREA =="AZU" ')
dataframe.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
1929,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1930,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1931,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1932,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1933,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL


In [53]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2361 entries, 1929 to 258326
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   SITUACAO_VOO                 2361 non-null   category
 1   MES_PARTIDA_PREVISTA         2361 non-null   int32   
 2   NOME_MES_PARTIDA_PREVISTA    2361 non-null   category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM  2361 non-null   string  
 4   NOME_AEROPORTO_ORIGEM        2361 non-null   string  
 5   SIGLA_ICAO_EMPRESA_AEREA     2361 non-null   string  
 6   NOME_EMPRESA                 2361 non-null   string  
dtypes: category(2), int32(1), string(4)
memory usage: 106.6 KB


In [54]:
dataframe[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]] = dataframe[['SITUACAO_VOO','NOME_MES_PARTIDA_PREVISTA' ]].astype('string')

In [55]:
df_gb = dataframe.groupby(['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_SITUACAO=('SITUACAO_VOO', 'count')
).sort_values(by=['SITUACAO_VOO','MES_PARTIDA_PREVISTA']).reset_index()
df_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_SITUACAO
0,CANCELADO,janeiro,1,17
1,CANCELADO,fevereiro,2,2
2,CANCELADO,março,3,2
3,CANCELADO,maio,5,9
4,CANCELADO,junho,6,13
5,CANCELADO,julho,7,4
6,CANCELADO,agosto,8,3
7,CANCELADO,setembro,9,11
8,CANCELADO,outubro,10,15
9,CANCELADO,novembro,11,10


In [56]:
df_gb['TOTAL_SITUACAO_MES_ANTERIOR'] = df_gb.groupby('SITUACAO_VOO')['TOTAL_SITUACAO'].shift(1)
df_gb.fillna(0, axis=1, inplace=True)
df_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR
0,CANCELADO,janeiro,1,17,0
1,CANCELADO,fevereiro,2,2,17.0
2,CANCELADO,março,3,2,2.0
3,CANCELADO,maio,5,9,2.0
4,CANCELADO,junho,6,13,9.0
5,CANCELADO,julho,7,4,13.0
6,CANCELADO,agosto,8,3,4.0
7,CANCELADO,setembro,9,11,3.0
8,CANCELADO,outubro,10,15,11.0
9,CANCELADO,novembro,11,10,15.0


In [57]:
def calcular_variacao(linha):
    try:
        total_atual = linha['TOTAL_SITUACAO']
        total_mes_anterior = linha['TOTAL_SITUACAO_MES_ANTERIOR']
        variacao = ((total_atual - total_mes_anterior) / total_mes_anterior) * 100
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)


In [58]:
df_gb['PERCENTUAL_VARIACAO'] = df_gb.apply(calcular_variacao, axis=1)
df_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
0,CANCELADO,janeiro,1,17,0,0.00
1,CANCELADO,fevereiro,2,2,17.0,-88.24
2,CANCELADO,março,3,2,2.0,0.00
3,CANCELADO,maio,5,9,2.0,350.00
4,CANCELADO,junho,6,13,9.0,44.44
5,CANCELADO,julho,7,4,13.0,-69.23
6,CANCELADO,agosto,8,3,4.0,-25.00
7,CANCELADO,setembro,9,11,3.0,266.67
8,CANCELADO,outubro,10,15,11.0,36.36
9,CANCELADO,novembro,11,10,15.0,-33.33


In [59]:
df_status_cancelado = df_gb.query('SITUACAO_VOO == "CANCELADO" and MES_PARTIDA_PREVISTA == 2')[['TOTAL_SITUACAO', 'TOTAL_SITUACAO_MES_ANTERIOR', 'PERCENTUAL_VARIACAO']]
df_status_cancelado

,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
1,2,17.0,-88.24


In [60]:
df_status_realizado = df_gb.query('SITUACAO_VOO == "REALIZADO" and MES_PARTIDA_PREVISTA == 2')[['TOTAL_SITUACAO', 'TOTAL_SITUACAO_MES_ANTERIOR', 'PERCENTUAL_VARIACAO']]
df_status_realizado

,TOTAL_SITUACAO,TOTAL_SITUACAO_MES_ANTERIOR,PERCENTUAL_VARIACAO
12,144,240.0,-40.0


In [61]:
fig = go.Figure(
    go.Indicator(
        mode='number+delta',
        value=df_status_realizado['TOTAL_SITUACAO'].values[0],
        delta={
            'position': 'top', 
            'reference': df_status_realizado['TOTAL_SITUACAO_MES_ANTERIOR'].values[0], 
            'relative': True,
        },
        title={
            'text': 'Total Situação Vôos Realizados'
        }
    )
)
fig.show()

- Composição das situação do voo filtre por aeroporto em empresa

In [62]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA'

]
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [63]:
dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"  and SIGLA_ICAO_EMPRESA_AEREA =="AZU"') #  and SITUACAO_VOO=="REALIZADO"
dataframe.head()

,SITUACAO_VOO,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,NOME_EMPRESA
1929,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1930,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1931,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1932,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL
1933,REALIZADO,1,janeiro,SBRP,LEITE LOPES,AZU,AZUL


In [64]:
dataframe[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']] = dataframe[['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA']].astype('string')

In [65]:
dataframe_gb = dataframe.groupby(['SITUACAO_VOO', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('NOME_MES_PARTIDA_PREVISTA', 'count')
).reset_index().sort_values(by=['MES_PARTIDA_PREVISTA'])
dataframe_gb

,SITUACAO_VOO,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_VOOS
3,CANCELADO,janeiro,1,17
15,REALIZADO,janeiro,1,240
2,CANCELADO,fevereiro,2,2
14,REALIZADO,fevereiro,2,144
19,REALIZADO,março,3,171
7,CANCELADO,março,3,2
11,REALIZADO,abril,4,194
6,CANCELADO,maio,5,9
18,REALIZADO,maio,5,212
5,CANCELADO,junho,6,13


In [66]:
dataframe_total = dataframe_gb.groupby([ 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOO_MES=('TOTAL_VOOS', 'sum')
).reset_index().sort_values(by=['MES_PARTIDA_PREVISTA'])
dataframe_total

,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_VOO_MES
4,janeiro,1,257
3,fevereiro,2,146
8,março,3,173
0,abril,4,194
7,maio,5,221
6,junho,6,203
5,julho,7,223
1,agosto,8,198
11,setembro,9,196
10,outubro,10,195


In [67]:

def obter_variacao(linha):
    try:
        total_voo_mes = linha['TOTAL_VOO_MES']
        total_voo = linha['TOTAL_VOOS']
        variacao = (total_voo * 100) / total_voo_mes
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)


In [68]:
dataframe_completo = dataframe_total.merge(dataframe_gb, how='inner', on='NOME_MES_PARTIDA_PREVISTA')
dataframe_completo = dataframe_completo[['NOME_MES_PARTIDA_PREVISTA', 'SITUACAO_VOO', 'TOTAL_VOOS', 'TOTAL_VOO_MES']]

In [69]:
dataframe_completo['VARIACAO_MENSAL'] = dataframe_completo.apply(obter_variacao, axis=1)
dataframe_completo

,NOME_MES_PARTIDA_PREVISTA,SITUACAO_VOO,TOTAL_VOOS,TOTAL_VOO_MES,VARIACAO_MENSAL
0,janeiro,CANCELADO,17,257,6.61
1,janeiro,REALIZADO,240,257,93.39
2,fevereiro,CANCELADO,2,146,1.37
3,fevereiro,REALIZADO,144,146,98.63
4,março,REALIZADO,171,173,98.84
5,março,CANCELADO,2,173,1.16
6,abril,REALIZADO,194,194,100.00
7,maio,CANCELADO,9,221,4.07
8,maio,REALIZADO,212,221,95.93
9,junho,CANCELADO,13,203,6.40


- Compare total de vôos realizado no fim de semana em relação por dia útil por mês

In [70]:
colunas = [
    'SITUACAO_VOO',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA',
    'INDICE_SEMANA_PARTIDA_PREVISTA',
    'DIA_DA_SEMANA_PARTIDA_PREVISTA'
    

]
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [71]:
dataframe = dataframe.query('SIGLA_ICAO_EMPRESA_AEREA == "AZU" and SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP" ')

In [72]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2361 entries, 1929 to 258326
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   SITUACAO_VOO                    2361 non-null   category
 1   MES_PARTIDA_PREVISTA            2361 non-null   int32   
 2   NOME_MES_PARTIDA_PREVISTA       2361 non-null   category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM     2361 non-null   string  
 4   NOME_AEROPORTO_ORIGEM           2361 non-null   string  
 5   SIGLA_ICAO_EMPRESA_AEREA        2361 non-null   string  
 6   NOME_EMPRESA                    2361 non-null   string  
 7   INDICE_SEMANA_PARTIDA_PREVISTA  2361 non-null   int32   
 8   DIA_DA_SEMANA_PARTIDA_PREVISTA  2361 non-null   category
dtypes: category(3), int32(2), string(4)
memory usage: 118.5 KB


In [73]:
dataframe['DIA_DA_SEMANA_PARTIDA_PREVISTA'] = dataframe['DIA_DA_SEMANA_PARTIDA_PREVISTA'].astype('string')

In [74]:
dataframe_gb = dataframe.groupby(['DIA_DA_SEMANA_PARTIDA_PREVISTA', 'INDICE_SEMANA_PARTIDA_PREVISTA']).agg(
    TOTAL_VOOS=('DIA_DA_SEMANA_PARTIDA_PREVISTA', 'count')
).reset_index()
dataframe_gb = dataframe_gb.sort_values(by=['INDICE_SEMANA_PARTIDA_PREVISTA'])
dataframe_gb = dataframe_gb[['DIA_DA_SEMANA_PARTIDA_PREVISTA', 'TOTAL_VOOS']]

In [75]:
dataframe_gb

,DIA_DA_SEMANA_PARTIDA_PREVISTA,TOTAL_VOOS
3,Segunda-feira,407
6,Terça-feira,325
1,Quarta-feira,320
2,Quinta-feira,324
4,Sexta-feira,310
5,Sábado,321
0,Domingo,354


In [76]:
fig = px.bar(dataframe_gb, x='DIA_DA_SEMANA_PARTIDA_PREVISTA', y='TOTAL_VOOS', text_auto='0')
fig.update_traces(
    textposition='outside'
)
fig
fig.show()

<p> Analise a FAIXA_ATRASO_PARTIDA por aeroporto e empresa por dia de semana e por mês </p>

In [77]:
colunas = [
    'FAIXA_ATRASO_PARTIDA',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'NOME_EMPRESA',
    'INDICE_SEMANA_PARTIDA_PREVISTA',
    'DIA_DA_SEMANA_PARTIDA_PREVISTA'
]
dataframe = pd.read_parquet(caminho_base, columns=colunas)

In [78]:
dataframe = dataframe.query('SIGLA_ICAO_EMPRESA_AEREA == "AZU" and SIGLA_ICAO_AEROPORTO_ORIGEM=="SBRP" and MES_PARTIDA_PREVISTA  == 2')
# dataframe = dataframe.query('SIGLA_ICAO_EMPRESA_AEREA == "AZU" and SIGLA_ICAO_AEROPORTO_ORIGEM=="SBRP" ')

In [79]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1938 to 237113
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   FAIXA_ATRASO_PARTIDA            146 non-null    category
 1   MES_PARTIDA_PREVISTA            146 non-null    int32   
 2   NOME_MES_PARTIDA_PREVISTA       146 non-null    category
 3   SIGLA_ICAO_AEROPORTO_ORIGEM     146 non-null    string  
 4   NOME_AEROPORTO_ORIGEM           146 non-null    string  
 5   SIGLA_ICAO_EMPRESA_AEREA        146 non-null    string  
 6   NOME_EMPRESA                    146 non-null    string  
 7   INDICE_SEMANA_PARTIDA_PREVISTA  146 non-null    int32   
 8   DIA_DA_SEMANA_PARTIDA_PREVISTA  146 non-null    category
dtypes: category(3), int32(2), string(4)
memory usage: 8.2 KB


In [80]:
dataframe[['FAIXA_ATRASO_PARTIDA', 'NOME_MES_PARTIDA_PREVISTA','DIA_DA_SEMANA_PARTIDA_PREVISTA' ]] = dataframe[['FAIXA_ATRASO_PARTIDA', 'NOME_MES_PARTIDA_PREVISTA','DIA_DA_SEMANA_PARTIDA_PREVISTA' ]].astype('string')

In [81]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 1938 to 237113
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   FAIXA_ATRASO_PARTIDA            146 non-null    string
 1   MES_PARTIDA_PREVISTA            146 non-null    int32 
 2   NOME_MES_PARTIDA_PREVISTA       146 non-null    string
 3   SIGLA_ICAO_AEROPORTO_ORIGEM     146 non-null    string
 4   NOME_AEROPORTO_ORIGEM           146 non-null    string
 5   SIGLA_ICAO_EMPRESA_AEREA        146 non-null    string
 6   NOME_EMPRESA                    146 non-null    string
 7   INDICE_SEMANA_PARTIDA_PREVISTA  146 non-null    int32 
 8   DIA_DA_SEMANA_PARTIDA_PREVISTA  146 non-null    string
dtypes: int32(2), string(7)
memory usage: 10.3 KB


In [82]:
df_gb = dataframe.groupby(['FAIXA_ATRASO_PARTIDA', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_FAIXA_ATRASO_SAIDA=('FAIXA_ATRASO_PARTIDA', 'count')
).reset_index()


In [83]:
def calcular_variacao(linha):
    try:
        total_atual = linha['TOTAL_FAIXA_ATRASO_SAIDA']
        total_mes_anterior = linha['TOTAL_FAIXA_ATRASO_PARTIDA_ANTERIOR']
        variacao = ((total_atual - total_mes_anterior) / total_mes_anterior) * 100
    except ZeroDivisionError:
        variacao = 0
    return round(variacao, 2)


In [84]:
dataframe_total = dataframe.groupby(['FAIXA_ATRASO_PARTIDA', 'MES_PARTIDA_PREVISTA', 'NOME_MES_PARTIDA_PREVISTA']).agg(
    TOTAL_VOO_MES=('FAIXA_ATRASO_PARTIDA', 'count')
).reset_index().sort_values(by=['MES_PARTIDA_PREVISTA'])
dataframe_total.head()

,FAIXA_ATRASO_PARTIDA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,TOTAL_VOO_MES
0,Atraso de até 30 minutos,2,fevereiro,9
1,Atraso de mais de 4 horas,2,fevereiro,2
2,No horário,2,fevereiro,135


In [85]:
fig = ff.create_table(dataframe_total)



fig.show()

- Variação de atraso Partida total por aeroporto em relação ao mês anterior (Análise por Flag)

In [86]:
colunas = [
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_EMPRESA',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'DIA_DA_SEMANA_PARTIDA_PREVISTA',
    'INDICE_SEMANA_PARTIDA_PREVISTA',
    'FAIXA_ATRASO_PARTIDA',
    'FLAG_ATRASO_PARTIDA',
    'FLAG_FIM_SEMANA_PARTIDA_PREVISTA'
]

In [87]:
dataframe = pd.read_parquet(caminho_base, columns=colunas)
dataframe.head()

,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_EMPRESA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,FAIXA_ATRASO_PARTIDA,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA
0,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
1,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
2,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
3,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
4,BAURU/AREALVA,AZU,SBAE,AZUL,2,fevereiro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana


In [88]:
dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"')
# dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP" and SIGLA_ICAO_EMPRESA_AEREA == "AZU"  and MES_PARTIDA_PREVISTA == 1')
dataframe.head()

,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_EMPRESA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,FAIXA_ATRASO_PARTIDA,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA
1929,LEITE LOPES,AZU,SBRP,AZUL,1,janeiro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1930,LEITE LOPES,AZU,SBRP,AZUL,1,janeiro,Segunda-feira,0,Atraso de até 30 minutos,Vôo no horário,Dia Útil
1931,LEITE LOPES,AZU,SBRP,AZUL,1,janeiro,Quarta-feira,2,No horário,Vôo Atrasado,Dia Útil
1932,LEITE LOPES,AZU,SBRP,AZUL,1,janeiro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1933,LEITE LOPES,AZU,SBRP,AZUL,1,janeiro,Segunda-feira,0,No horário,Vôo Atrasado,Dia Útil


In [89]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4955 entries, 1929 to 710551
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   NOME_AEROPORTO_ORIGEM             4955 non-null   string  
 1   SIGLA_ICAO_EMPRESA_AEREA          4955 non-null   string  
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       4955 non-null   string  
 3   NOME_EMPRESA                      4955 non-null   string  
 4   MES_PARTIDA_PREVISTA              4955 non-null   int32   
 5   NOME_MES_PARTIDA_PREVISTA         4955 non-null   category
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    4955 non-null   category
 7   INDICE_SEMANA_PARTIDA_PREVISTA    4955 non-null   int32   
 8   FAIXA_ATRASO_PARTIDA              4955 non-null   category
 9   FLAG_ATRASO_PARTIDA               4955 non-null   string  
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  4955 non-null   string  
dtypes: category(3), int32(2), string(6)
memory usage: 325.1 

In [90]:
dataframe['NOME_MES_PARTIDA_PREVISTA'] = dataframe['NOME_MES_PARTIDA_PREVISTA'].astype('string')

In [91]:
dataframe = dataframe.groupby(['FLAG_ATRASO_PARTIDA', 'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_PREVISTA']).agg(
    TOTAL_ATRASO=('FLAG_ATRASO_PARTIDA', 'count')
).reset_index()
dataframe

,FLAG_ATRASO_PARTIDA,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_ATRASO
0,Vôo Atrasado,abril,4,275
1,Vôo Atrasado,agosto,8,252
2,Vôo Atrasado,dezembro,12,223
3,Vôo Atrasado,fevereiro,2,201
4,Vôo Atrasado,janeiro,1,227
5,Vôo Atrasado,julho,7,387
6,Vôo Atrasado,junho,6,251
7,Vôo Atrasado,maio,5,307
8,Vôo Atrasado,março,3,229
9,Vôo Atrasado,novembro,11,215


In [92]:
dataframe = dataframe.sort_values(by=['FLAG_ATRASO_PARTIDA','MES_PARTIDA_PREVISTA', ])

In [93]:
dataframe['TOTAL_ATRASO_DESLOCADO'] = dataframe.groupby('FLAG_ATRASO_PARTIDA')['TOTAL_ATRASO'].shift(1)
dataframe.fillna(0, axis=1, inplace=True)
dataframe

,FLAG_ATRASO_PARTIDA,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_ATRASO,TOTAL_ATRASO_DESLOCADO
4,Vôo Atrasado,janeiro,1,227,0
3,Vôo Atrasado,fevereiro,2,201,227.0
8,Vôo Atrasado,março,3,229,201.0
0,Vôo Atrasado,abril,4,275,229.0
7,Vôo Atrasado,maio,5,307,275.0
6,Vôo Atrasado,junho,6,251,307.0
5,Vôo Atrasado,julho,7,387,251.0
1,Vôo Atrasado,agosto,8,252,387.0
11,Vôo Atrasado,setembro,9,235,252.0
10,Vôo Atrasado,outubro,10,223,235.0


In [94]:
dataframe['VARIACAO_PERCENTUAL'] = (((dataframe['TOTAL_ATRASO_DESLOCADO'] - dataframe['TOTAL_ATRASO']) / dataframe['TOTAL_ATRASO']) * 100)
dataframe

,FLAG_ATRASO_PARTIDA,NOME_MES_PARTIDA_PREVISTA,MES_PARTIDA_PREVISTA,TOTAL_ATRASO,TOTAL_ATRASO_DESLOCADO,VARIACAO_PERCENTUAL
4,Vôo Atrasado,janeiro,1,227,0,-100.0
3,Vôo Atrasado,fevereiro,2,201,227.0,12.935323
8,Vôo Atrasado,março,3,229,201.0,-12.227074
0,Vôo Atrasado,abril,4,275,229.0,-16.727273
7,Vôo Atrasado,maio,5,307,275.0,-10.423453
6,Vôo Atrasado,junho,6,251,307.0,22.310757
5,Vôo Atrasado,julho,7,387,251.0,-35.142119
1,Vôo Atrasado,agosto,8,252,387.0,53.571429
11,Vôo Atrasado,setembro,9,235,252.0,7.234043
10,Vôo Atrasado,outubro,10,223,235.0,5.381166


In [95]:
dataframe[dataframe['NOME_MES_PARTIDA_PREVISTA'] == 'dezembro'][['FLAG_ATRASO_PARTIDA','TOTAL_ATRASO', 'VARIACAO_PERCENTUAL']]

,FLAG_ATRASO_PARTIDA,TOTAL_ATRASO,VARIACAO_PERCENTUAL
2,Vôo Atrasado,223,-3.587444
14,Vôo no horário,190,-9.473684


- Variação de atraso Partida total por aeroporto em relação ao mês anterior (Análise por Flag)

- Variação de FAIXA_ATRASO_PARTIDA por aeroporto em relação ao mês anterior 

<p> O volume de atraso de saída no fim de semana, é maior ou menor em relação aos outros dias uteis</p>

In [96]:
base_completa.columns

Index(['NOME_AEROPORTO_ORIGEM', 'CIDADE_AEROPORTO_ORIGEM',
       'UF_AEROPORTO_ORIGEM', 'SIGLA_ICAO_EMPRESA_AEREA', 'NUMERO_VOO',
       'CODIGO_TIPO_LINHA', 'SIGLA_ICAO_AEROPORTO_ORIGEM',
       'SIGLA_ICAO_AEROPORTO_DESTINO', 'PARTIDA_PREVISTA', 'PARTIDA_REAL',
       'CHEGADA_PREVISTA', 'CHEGADA_REAL', 'SITUACAO_VOO', 'SIGLA_OACI',
       'NOME_AEROPORTO_DESTINO', 'CIDADE_AEROPORTO_DESTINO',
       'UF_AEROPORTO_DESTINO', 'NOME_EMPRESA', 'MES_PARTIDA_PREVISTA',
       'NOME_MES_PARTIDA_PREVISTA', 'MES_PARTIDA_REAL',
       'NOME_MES_PARTIDA_REAL', 'MES_CHEGADA_PREVISTA',
       'NOME_MES_CHEGADA_PREVISTA', 'MES_CHEGADA_REAL',
       'NOME_MES_CHEGADA_REAL', 'MINUTOS_ATRASADO_PARTIDA',
       'MINUTOS_ATRASADO_CHEGADA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA',
       'INDICE_SEMANA_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_REAL',
       'INDICE_SEMANA_PARTIDA_REAL', 'DIA_DA_SEMANA_CHEGADA_REAL',
       'DIA_DA_SEMANA_CHEGADA_PREVISTA', 'INDICE_SEMANA_CHEGADA_REAL',
       'INDICE_SEMANA_CH

In [97]:
colunas = [
    'NOME_AEROPORTO_ORIGEM',
    'SIGLA_ICAO_EMPRESA_AEREA',
    'SIGLA_ICAO_AEROPORTO_ORIGEM',
    'NOME_EMPRESA',
    'MES_PARTIDA_PREVISTA',
    'NOME_MES_PARTIDA_PREVISTA',
    'DIA_DA_SEMANA_PARTIDA_PREVISTA',
    'INDICE_SEMANA_PARTIDA_PREVISTA',
    'FAIXA_ATRASO_PARTIDA',
    'FLAG_ATRASO_PARTIDA',
    'FLAG_FIM_SEMANA_PARTIDA_PREVISTA'
]

In [138]:
def obter_variacao_incidencia(
        mes: int, 
        sigla_aeroporto: str, 
        path_coluna_analise: str,
        sigla_empresa_aerea: str = None,
    ) -> pd.DataFrame:
    if sigla_empresa_aerea is None:
        query = f'SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}"  and MES_{path_coluna_analise}_PREVISTA == {mes}'
    else:
        query = f' SIGLA_ICAO_AEROPORTO_ORIGEM == "{sigla_aeroporto}" ' \
                f' and SIGLA_ICAO_EMPRESA_AEREA == "{sigla_empresa_aerea}" '\
                f' and MES_{path_coluna_analise}_PREVISTA == {mes} '
    colunas = [
        'NOME_AEROPORTO_ORIGEM',
        'SIGLA_ICAO_EMPRESA_AEREA',
        'SIGLA_ICAO_AEROPORTO_ORIGEM',
        'NOME_EMPRESA',
        f'MES_{path_coluna_analise}_PREVISTA',
        f'NOME_MES_{path_coluna_analise}_PREVISTA',
        f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA',
        f'INDICE_SEMANA_{path_coluna_analise}_PREVISTA',
        f'FAIXA_ATRASO_{path_coluna_analise}',
        f'FLAG_ATRASO_{path_coluna_analise}',
        f'FLAG_FIM_SEMANA_{path_coluna_analise}_PREVISTA'
    ]
    dataframe = pd.read_parquet(caminho_base, columns=colunas)
    dataframe = dataframe.query(query)
    dataframe[
        [
            f'NOME_MES_{path_coluna_analise}_PREVISTA', 
            f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA', 
            f'FAIXA_ATRASO_{path_coluna_analise}'
        ]
    ] = dataframe[
        [
            f'NOME_MES_{path_coluna_analise}_PREVISTA', 
            f'DIA_DA_SEMANA_{path_coluna_analise}_PREVISTA', 
            f'FAIXA_ATRASO_{path_coluna_analise}'
        ]
    ].astype('string')
    dataframe = dataframe.groupby(
        [
            f'FLAG_ATRASO_{path_coluna_analise}', 
            f'FLAG_FIM_SEMANA_{path_coluna_analise}_PREVISTA'
        ]
    ).agg(
        TOTAL_INCIDENCIA=(f'FLAG_ATRASO_{path_coluna_analise}', 'count')
    ).reset_index()
    dataframe[f'TOTAL_INCIDENCIA_GERAL_{path_coluna_analise}'] = dataframe.groupby(f'FLAG_ATRASO_{path_coluna_analise}')['TOTAL_INCIDENCIA'].transform('sum')
    dataframe[f'PERCENTUAL_INCIDENCIA_{path_coluna_analise}'] = round((dataframe['TOTAL_INCIDENCIA'] * 100) / dataframe[f'TOTAL_INCIDENCIA_GERAL_{path_coluna_analise}'], 2)
    return dataframe

In [139]:
obter_variacao_incidencia(mes=1, sigla_aeroporto='SBRP', path_coluna_analise='PARTIDA')

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA,TOTAL_INCIDENCIA_GERAL_PARTIDA,PERCENTUAL_INCIDENCIA_PARTIDA
0,Vôo Atrasado,Dia Útil,160,227,70.48
1,Vôo Atrasado,Fim de Semana,67,227,29.52
2,Vôo no horário,Dia Útil,346,378,91.53
3,Vôo no horário,Fim de Semana,32,378,8.47


In [98]:
dataframe = pd.read_parquet(caminho_base, columns=colunas)
dataframe.head()

,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_EMPRESA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,FAIXA_ATRASO_PARTIDA,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA
0,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
1,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
2,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
3,BAURU/AREALVA,AZU,SBAE,AZUL,1,janeiro,Domingo,6,No horário,Vôo Atrasado,Fim de Semana
4,BAURU/AREALVA,AZU,SBAE,AZUL,2,fevereiro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana


In [99]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717357 entries, 0 to 717356
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype   
---  ------                            --------------   -----   
 0   NOME_AEROPORTO_ORIGEM             717357 non-null  string  
 1   SIGLA_ICAO_EMPRESA_AEREA          717357 non-null  string  
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       717357 non-null  string  
 3   NOME_EMPRESA                      717357 non-null  string  
 4   MES_PARTIDA_PREVISTA              717357 non-null  int32   
 5   NOME_MES_PARTIDA_PREVISTA         717357 non-null  category
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    717357 non-null  category
 7   INDICE_SEMANA_PARTIDA_PREVISTA    717357 non-null  int32   
 8   FAIXA_ATRASO_PARTIDA              717357 non-null  category
 9   FLAG_ATRASO_PARTIDA               717357 non-null  string  
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  717357 non-null  string  
dtypes: category(3), int32(2), string(6)
mem

- Incidencia de atraso total por aeroporto (Análise por Flag)

In [100]:
dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP"  and MES_PARTIDA_PREVISTA == 12')
# dataframe = dataframe.query('SIGLA_ICAO_AEROPORTO_ORIGEM == "SBRP" and SIGLA_ICAO_EMPRESA_AEREA == "AZU"  and MES_PARTIDA_PREVISTA == 1')
dataframe.head()

,NOME_AEROPORTO_ORIGEM,SIGLA_ICAO_EMPRESA_AEREA,SIGLA_ICAO_AEROPORTO_ORIGEM,NOME_EMPRESA,MES_PARTIDA_PREVISTA,NOME_MES_PARTIDA_PREVISTA,DIA_DA_SEMANA_PARTIDA_PREVISTA,INDICE_SEMANA_PARTIDA_PREVISTA,FAIXA_ATRASO_PARTIDA,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA
1992,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1993,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1994,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Domingo,6,No horário,Vôo no horário,Fim de Semana
1995,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana
1996,LEITE LOPES,AZU,SBRP,AZUL,12,dezembro,Sábado,5,No horário,Vôo Atrasado,Fim de Semana


In [101]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 1992 to 705207
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   NOME_AEROPORTO_ORIGEM             413 non-null    string  
 1   SIGLA_ICAO_EMPRESA_AEREA          413 non-null    string  
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       413 non-null    string  
 3   NOME_EMPRESA                      413 non-null    string  
 4   MES_PARTIDA_PREVISTA              413 non-null    int32   
 5   NOME_MES_PARTIDA_PREVISTA         413 non-null    category
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    413 non-null    category
 7   INDICE_SEMANA_PARTIDA_PREVISTA    413 non-null    int32   
 8   FAIXA_ATRASO_PARTIDA              413 non-null    category
 9   FLAG_ATRASO_PARTIDA               413 non-null    string  
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  413 non-null    string  
dtypes: category(3), int32(2), string(6)
memory usage: 28.0 KB

In [102]:
dataframe[['NOME_MES_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA', 'FAIXA_ATRASO_PARTIDA']] = dataframe[['NOME_MES_PARTIDA_PREVISTA', 'DIA_DA_SEMANA_PARTIDA_PREVISTA', 'FAIXA_ATRASO_PARTIDA']].astype('string')

In [103]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 1992 to 705207
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   NOME_AEROPORTO_ORIGEM             413 non-null    string
 1   SIGLA_ICAO_EMPRESA_AEREA          413 non-null    string
 2   SIGLA_ICAO_AEROPORTO_ORIGEM       413 non-null    string
 3   NOME_EMPRESA                      413 non-null    string
 4   MES_PARTIDA_PREVISTA              413 non-null    int32 
 5   NOME_MES_PARTIDA_PREVISTA         413 non-null    string
 6   DIA_DA_SEMANA_PARTIDA_PREVISTA    413 non-null    string
 7   INDICE_SEMANA_PARTIDA_PREVISTA    413 non-null    int32 
 8   FAIXA_ATRASO_PARTIDA              413 non-null    string
 9   FLAG_ATRASO_PARTIDA               413 non-null    string
 10  FLAG_FIM_SEMANA_PARTIDA_PREVISTA  413 non-null    string
dtypes: int32(2), string(9)
memory usage: 35.5 KB


In [104]:
dataframe = dataframe.groupby(['FLAG_ATRASO_PARTIDA', 'FLAG_FIM_SEMANA_PARTIDA_PREVISTA' ]).agg(
    TOTAL_INCIDENCIA=('FLAG_ATRASO_PARTIDA', 'count')
).reset_index()


In [105]:
dataframe

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA
0,Vôo Atrasado,Dia Útil,151
1,Vôo Atrasado,Fim de Semana,72
2,Vôo no horário,Dia Útil,150
3,Vôo no horário,Fim de Semana,40


In [106]:
dataframe.groupby('FLAG_ATRASO_PARTIDA')['TOTAL_INCIDENCIA'].transform('sum')

0    223
1    223
2    190
3    190
Name: TOTAL_INCIDENCIA, dtype: int64

In [107]:
dataframe['TOTAL_INCIDENCIA_GERAL'] = dataframe.groupby('FLAG_ATRASO_PARTIDA')['TOTAL_INCIDENCIA'].transform('sum')

In [109]:
dataframe['PERCENTUAL_INCIDENCIA'] = round((dataframe['TOTAL_INCIDENCIA'] * 100) / dataframe['TOTAL_INCIDENCIA_GERAL'], 2)

In [110]:
dataframe

,FLAG_ATRASO_PARTIDA,FLAG_FIM_SEMANA_PARTIDA_PREVISTA,TOTAL_INCIDENCIA,TOTAL_INCIDENCIA_GERAL,PERCENTUAL_INCIDENCIA
0,Vôo Atrasado,Dia Útil,151,223,67.71
1,Vôo Atrasado,Fim de Semana,72,223,32.29
2,Vôo no horário,Dia Útil,150,190,78.95
3,Vôo no horário,Fim de Semana,40,190,21.05


In [111]:
fig = px.bar(
    dataframe,
    x='TOTAL_INCIDENCIA',
    y='FLAG_FIM_SEMANA_PARTIDA_PREVISTA',
    color='FLAG_ATRASO_PARTIDA',
    barmode='group'
)
fig.show()

<p> Pode ser análisado a incidencia por aeroporto também</p>
<p>Detalhe por semana também</p>
<p>Eu tenho 3000 vôos que partiram no horário 50%(1500) da empresa A e 50% da empresa B</p>

<p> - Analise a FAIXA_ATRASO_CHEGADA por aeroporto e empresa por dia de semana e por mês </p>
<p> O volume de atraso no fim de semana, é maior ou menor em relação ao dia atual</p>
<p> Pode ser análisado a incidencia por aeroporto também</p>
<p>Detalhe por semana também</p>
<p>Eu tenho 3000 vôos que partiram no horário 50%(1500) da empresa A e 50% da empresa B</p>


<p> Ranking por FAIXA_ATRASO_CHEGADA selecionando aeroporto  por mês e empresa</p>

<p> Ranking por FAIXA_ATRASO_SAIDA selecionando aeroporto em empresa por mês</p>

<p> Quantifique a porcentagem por FAIXA_ATRASO_CHEGADA selecionando aeroportoa por mês</p>

<p> Quantifique a porcentagem por FAIXA_ATRASO_PARTIFA selecionando aeroporto  por mês</p>